# Abstract

# Environment

In [ ]:
import json
import os.path as path
import re

In [ ]:
import jwst.associations as associations
from jwst.associations import load_asn
from jwst.associations.lib.rules_level2_base import _REGEX_LEVEL2A

# Library

In [ ]:
def lv2_v1_to_v2(asn):
    new_asn = dict(asn)
    products = []
    members_to_add = []
    for member in asn['members']:
        product = {
            'name': product_name(member['expname'])
        }
        new_members = []
        new_members.append({
            'expname': member['expname'],
            'exptype': member['exptype']
        })
        for bkg in member.get('bkgexps', []):
            new_members.append({
                'expname': bkg['expname'],
                'exptype': 'BACKGROUND'
            })
            members_to_add.append({
                'expname': bkg['expname'],
                'exptype': 'SCIENCE'
            })
        product['members'] = new_members
        products.append(product)
    
    for new_member in members_to_add:
        product = {
            'name': product_name(new_member['expname'])
        }
        product['members'] = [{
            'expname': new_member['expname'],
            'exptype': 'SCIENCE'
        }]
        products.append(product)
        
    new_asn['products'] = products
    del new_asn['members']
    return new_asn
    

In [ ]:
def product_name(expname):
    name = path.basename(expname.lower())
    match = re.match(_REGEX_LEVEL2A, name)
    if match:
        return match.groupdict()['path']
    else:
        return name

# Main

In [ ]:
with open('data/asn_level2_v1.json') as fp:
    lv2_v1 = json.load(fp)

In [ ]:
lv2_v1

In [ ]:
lv2_v2 = lv2_v1_to_v2(lv2_v1)

In [ ]:
lv2_v2

In [ ]:
with open('/tmp/level2b_asn.json', 'w') as fp:
    json.dump(lv2_v2, fp)

In [ ]:
with open('/tmp/level2b_asn.json') as fp:
    asn = load_asn(fp)

In [ ]:
asn

In [ ]:
asn['products']

In [ ]:
asn['products'][0]['members']